In [ ]:
# Install fork of spint with modifications to accept multiple cost variables
# !pip install git+https://github.com/chesterharvey/spint

In [62]:
from modeling import *
from feature_dev import *
from utils import configs

%load_ext autoreload
%autoreload 2

df = develop_features()
# df.columns.tolist()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
8190 initial records
8190 complete records from load_travel_time_and_fares
8190 complete records from load_track_miles
8190 complete records from load_am_ridership
8190 complete records from load_pm_ridership
8190 complete records from load_off_peak_ridership
8190 complete records from calculate_total_ridership_and_parking_users
8190 complete records from load_bus_travel_time
8190 complete records from load_am_auto_travel_time
8190 complete records from load_pm_auto_travel_time
8190 complete records from load_off_peak_auto_travel_time
8190 complete records from load_m25_stations
8190 complete records from load_median_hh_income
8190 complete records from load_nine_to_five_jobs
8190 complete records from load_night_and_weekend_jobs
8190 complete records from load_school_jobs
8190 complete records from load_sop
8190 complete records from calculate_interactions


In [87]:
flows = 'total_ridership_od'
o_vars = [
    'trains_per_hr_peak_o',
    'proportion_houses_o',
    'total_households_within_half_mi_o',
    'parking_capacity_o',
    'bus_line_count_o',
    'bus_stop_count_o',
    'total_jobs_within_half_mi_o',
    'miles_to_core_o',
    'terminal_dummy_2023_o',
    'm25_station_o',
    'median_hh_income_o',
    'nine_to_five_jobs_o',
    'night_weekend_jobs_o',
    'school_jobs_o',
    # 'sop_form_norm_o',
    # 'sop_dens_norm_o',
    # 'sop_prox_norm_o',
    # 'sop_conn_norm_o',
    # 'sop_park_norm_o',
    # 'sop_peds_norm_o',
    # 'sop_safe_norm_o',
    # 'sop_traf_norm_o',
    # 'sop_aest_norm_o',
    # 'sop_recr_norm_o',
    'sop_totl_norm_o',
]
d_vars = [
    'trains_per_hr_peak_d',
    'proportion_houses_d',
    'total_households_within_half_mi_d',
    'parking_capacity_d',
    'bus_line_count_d',
    'bus_stop_count_d',
    'total_jobs_within_half_mi_d',
    'miles_to_core_d',
    'terminal_dummy_2023_d',
    'm25_station_d',
    'median_hh_income_d',
    'nine_to_five_jobs_d',
    'night_weekend_jobs_d',
    'school_jobs_d',
    # 'sop_form_norm_d',
    # 'sop_dens_norm_d',
    # 'sop_prox_norm_d',
    # 'sop_conn_norm_d',
    # 'sop_park_norm_d',
    # 'sop_peds_norm_d',
    # 'sop_safe_norm_d',
    # 'sop_traf_norm_d',
    # 'sop_aest_norm_d',
    # 'sop_recr_norm_d',
    'sop_totl_norm_d',
]
od_vars = [
    'composite_miles_od',
    # 'peak_fare_od',
    # 'off_peak_fare_od',
    'sd_fare_od',
    'travel_time_od',
    'track_miles_od',
    # 'am_parking_user_od',
    # 'pm_parking_user_od',
    # 'off_peak_parking_user_od',
    'total_parking_user_od',
    'bus_tt_od',
    'bus_transfers_od',
    'am_auto_tt_od',
    # 'pm_auto_tt_od',
    # 'off_peak_auto_tt_od',
    'm25_station_o*m25_station_d',
]

# flows, o_vars, d_vars, od_vars = inputs(df, flows, o_vars, d_vars, od_vars)

estimate_models(df, flows, o_vars, d_vars, od_vars)

Gravity             OLS       
                                         coef    P       coef      P
const                                  -12.41  0.0     478.23    0.0
trains_per_hr_peak_o                     0.21  0.0       1.42    0.0
proportion_houses_o                     -0.27  0.0     -18.38  0.042
total_households_within_half_mi_o        0.63  0.0       0.03    0.0
parking_capacity_o                       -0.0  0.0      -0.03  0.003
bus_line_count_o                          0.1  0.0      -0.44  0.281
bus_stop_count_o                        -0.06  0.0      -0.05  0.924
total_jobs_within_half_mi_o             -0.04  0.0       -0.0  0.092
miles_to_core_o                          0.01  0.0     -13.62    0.0
terminal_dummy_2023_o                    0.01  0.0       3.81  0.896
m25_station_o                            0.04  0.0     181.12    0.0
median_hh_income_o                       0.05  0.0       -0.0  0.964
nine_to_five_jobs_o                      0.02  0.0        0.0  0.214
night_weekend_jobs_o                     0.02  0.0       0.02    0.0
school_jobs_o                            0.01  0.0       -0.0  0.922
sop_totl_norm_o                          0.84  0.0      -0.74  0.578
trains_per_hr_peak_d                     0.13  0.0       1.36    0.0
proportion_houses_d                     -0.28  0.0     -24.06  0.008
total_households_within_half_mi_d        0.65  0.0       0.04    0.0
parking_capacity_d                       -0.0  0.0      -0.03    0.0
bus_line_count_d                         0.08  0.0       -0.7  0.083
bus_stop_count_d                        -0.05  0.0       0.17  0.773
total_jobs_within_half_mi_d             -0.03  0.0        0.0   0.64
miles_to_core_d                          0.01  0.0     -12.76    0.0
terminal_dummy_2023_d                    0.02  0.0       3.22  0.912
m25_station_d                            0.04  0.0     200.58    0.0
median_hh_income_d                       0.09  0.0        0.0  0.801
nine_to_five_jobs_d                      0.01  0.0       -0.0  0.466
night_weekend_jobs_d                     0.01  0.0       0.02    0.0
school_jobs_d                            0.01  0.0       -0.0  0.104
sop_totl_norm_d                          1.13  0.0       1.72   0.19
composite_miles_od                       0.24  0.0      29.25    0.0
sd_fare_od                              -1.16  0.0    -209.37    0.0
travel_time_od                          -0.13  0.0     -40.67    0.0
track_miles_od                           0.13  0.0      84.18    0.0
total_parking_user_od                     0.0  0.0       1.62    0.0
bus_tt_od                                 0.0  0.0       0.63  0.003
bus_transfers_od                        -0.07  0.0     -44.58    0.0
am_auto_tt_od                             0.0  0.0       0.07  0.923
m25_station_o*m25_station_d              -0.4  0.0     325.51    0.0
AIC                                1288432.37       122308.63       
pseudoR2                                 0.72                       
R2                                                       0.44